In [ ]:
%reload_ext autoreload
%autoreload 2

import duckdb as db
import polars as pl
from database_etl.definitions import DB_PATH, RAW_DATA_LIB

pl.Config.set_fmt_str_lengths(9999)
pl.Config.set_tbl_rows(9999)
con = db.connect(DB_PATH)

image_files = list(RAW_DATA_LIB.glob("*.D/extract_*/data.parquet"))
len(image_files)

In [ ]:
con.sql(
    """--sql
show tables
"""
).pl()

In [ ]:
con.sql(
    """--sql
select * from image_stats limit 5
"""
).pl()

In [ ]:
con.sql(
    """--sql
select distinct mins_max from image_stats
"""
).pl()

In [ ]:
con.sql(
    """--sql
select
    *
from
    image_stats
join
    chm
on
    image_stats.pk = chm.pk
where
    mins_max < 20
"""
).pl()

# Creating Excluded Table and Adding Sample 20.


In [ ]:
con.sql(
    """--sql
create or replace table excluded (
    pk integer primary key references chm(pk),
    samplecode varchar unique not null,
    reason varchar not null
);
insert into excluded
    select
        pk,
        samplecode,
        'aborted run' as reason
    from
        chm
    where
        pk = 61;
create or replace view inc_chm as
    select
        *
    from
        chm
    anti join
        excluded
    on
        excluded.pk = chm.pk;
select
    *
from
    inc_chm
where
    pk = 61
or
    pk = 62
"""
).pl()

In [ ]:
con.sql(
    """--sql
describe excluded
"""
).pl()

# Creating Included Image Stats (`inc_img_stats`)


In [ ]:
# con.sql(
#     """--sql
# create or replace view inc_img_stats as
#     select
#         *
#     from
#         image_stats ist
#     anti join
#         excluded exc
#     on
#         ist.pk = exc.pk;
# select
#     *
# from
#     inc_img_stats
# limit
#     5;
# """
# ).pl()


In [ ]:
# con.sql(
#     """--sql
# select
#     distinct mins_max
# from
#     inc_img_stats
# """
# ).pl().pipe(display)


In [ ]:
con.sql(
    """--sql
describe inc_img_stats
"""
).pl()

So when accounting for the aborted run pk = 61, all runs have the same end time - 52 mins, until the 3rd significant figure. Considering that it is one observation every 0.4 seconds, then only the first two significant figures are relevent, and thus they are the same. I presume that rounding to the second signifiant figure will make all time labels the same - this is easier than resampling.

In [ ]:
paths = con.sql(
    """--sql
select
    path
from
    inc_img_stats
"""
).fetchall()

paths[0:5]

In [ ]:
paths[0][0]

In [ ]:
import pandas as pd

df = pd.read_parquet(paths[0][0])
df

In [ ]:
pl.read_parquet(
    "/Users/jonathan/mres_thesis/database_etl/database_etl/data/raw_uv/114.D/extract_2024-09-18T212559/data.parquet"
)

In [ ]:
df = pl.read_parquet(paths[0][0])
df